In [ ]:
import numpy as np
from pyfeti.src.feti_solver import SerialFETIsolver
from scipy.sparse import linalg

K1 = np.array([[4, 0, 0, 0],
               [0, 4, -1, -2],
               [0, -1, 4, -1],
               [0, -2, -1, 4]])

K2 = K3 = K4 = np.array([[4, -1, -2, -1],
                    [-1, 4, -1, -2],
                    [-2, -1, 4, -1],
                    [-1, -2, -1, 4]])


q0 = 10.0
q1 = np.array([0.,0.,0.,0.])
q2 = np.array([0.,0.,0.,0.])
q3 = np.array([0.,0.,0.,0.])
q4 = np.array([0.,0.,1.0,0.0])

B12 =  np.array([[0,1,0,0],
                 [0,0,1,0]])

B13 = np.array([[0,0,1,0],
                [0,0,0,1]])

B14 = np.array([[0,0,1,0]])

B21 =  np.array([[-1,0,0,0],
                 [0,0,0,-1]])

B23 = np.array([[0,0,0,1]])

B24 = np.array([[0,0,1,0],
                [0,0,0,1]])


B31 = np.array([[0,-1,0,0],
                [-1,0,0,0]])

B32 = np.array([[0,-1,0,0]])

B34 = np.array([[0,1,0,0],
                [0,0,1,0]])

B41 = np.array([[-1,0,0,0]])

B42 = np.array([[0,-1,0,0],
                [-1,0,0,0]])

B43 = np.array([[-1,0,0,0],
                [ 0,0,0,-1]])


# Using PyFETI to solve the probrem described above
K_dict = {1:K1,2:K2, 3:K3, 4:K4}
B_dict = {1 : {(1,2) : B12, (1,3) : B13, (1,4) : B14}, 
          2 : {(2,1) : B21, (2,4) : B24,(2,3) : B23}, 
          3 : {(3,1) : B31, (3,4) : B34, (3,2) : B32}, 
          4 : {(4,2) : B42, (4,3) : B43, (4,1) : B41}}

q_dict = {1:q1 ,2:q2, 3:q3, 4:q4}
solver_obj = SerialFETIsolver(K_dict,B_dict,q_dict)

In [ ]:

solution_obj = solver_obj.solve()
T_feti = solution_obj.displacement
lambda_ = solution_obj.interface_lambda
alpha =  solution_obj.alpha

In [ ]:
T_feti

In [ ]:
lambda_

In [ ]:
alpha

In [ ]:
L = solver_obj.manager.assemble_global_L()
Lexp = solver_obj.manager.assemble_global_L_exp()
B = solver_obj.manager.assemble_global_B()
K, f = solver_obj.manager.assemble_global_K_and_f()
R = solver_obj.manager.assemble_global_kernel()
e = solver_obj.manager.assemble_e()
G = solver_obj.manager.assemble_G()
GGT_inv = np.linalg.inv(G.dot(G.T))
P = np.eye(B.shape[0]) - (G.T.dot(GGT_inv)).dot(G)


In [ ]:
R.shape

In [4]:
f_primal = L.dot(f)
K_primal = L.dot(K.dot(Lexp))
T_primal = np.linalg.solve(K_primal,f_primal)

print(K_primal.shape)

T_primal

(9, 9)


array([0.        , 0.23333333, 0.26666667, 0.23333333, 0.26666667,
       0.3       , 0.3       , 0.26666667, 0.53333333])

In [5]:
K_primal.dot(T_primal)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.11022302e-16,
       -2.22044605e-16,  0.00000000e+00, -1.11022302e-16,  0.00000000e+00,
        1.00000000e+00])

In [6]:
T_dual = Lexp.dot(T_primal)

T_dual

array([0.        , 0.23333333, 0.26666667, 0.23333333, 0.23333333,
       0.26666667, 0.3       , 0.26666667, 0.23333333, 0.26666667,
       0.3       , 0.26666667, 0.26666667, 0.3       , 0.53333333,
       0.3       ])

In [7]:
B.dot(T_dual)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [8]:
B.shape

(10, 16)

In [9]:
G

array([[-0.5, -0.5,  0. ,  0. ,  0. ,  0.5,  0.5,  0.5,  0. ,  0. ],
       [ 0. ,  0. , -0.5, -0.5,  0. , -0.5,  0. ,  0. ,  0.5,  0.5],
       [ 0. ,  0. ,  0. ,  0. , -0.5,  0. , -0.5, -0.5, -0.5, -0.5]])

In [10]:
B.dot(T_dual)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
K_inv = np.linalg.pinv(K.A)

F = B@K_inv@B.T
d = B@K_inv@f

np.allclose(K.A, np.dot(K.A, np.dot(K_inv, K.A)))

True

In [12]:
np.allclose(K_inv, K_inv.T,atol=1e-10)

True

In [13]:
lambda_im = G.T.dot(GGT_inv).dot(e)

Fp = P.T.dot(F.dot(P))
dp = P.T.dot(d)
lambda_ker, info = linalg.cg(Fp,dp,M=P)
lambda_cg = lambda_im + lambda_ker
r = d - F.dot(lambda_cg)
alpha = GGT_inv.dot(G.dot(r))

In [14]:
lambda_cg

array([-1.57291667e-01, -1.63541667e-01, -1.63541667e-01, -1.57291667e-01,
       -3.58333333e-01, -2.34249624e-17, -1.26041667e-01, -1.94791667e-01,
       -1.94791667e-01, -1.26041667e-01])

In [15]:
Fp.dot(lambda_ker) - dp

array([-1.60461922e-17, -1.64798730e-17, -3.03576608e-18, -5.20417043e-18,
       -1.21430643e-17,  1.12757026e-17,  3.46944695e-18,  3.46944695e-18,
       -1.04083409e-17, -3.46944695e-18])

In [16]:
lambda_


NameError: name 'lambda_' is not defined

In [18]:
alpha

array([-0.53721065, -0.53721065, -0.72528935])

In [19]:
R

<3x16 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Column format>

In [17]:
T_cg = K_inv@(f - B.T@lambda_cg) + R.dot(alpha)

ValueError: dimension mismatch

In [ ]:
T_feti = K_inv@(f - B.T@lambda_)

In [ ]:
T_cg

In [ ]:
T_dual

In [ ]:
K.dot(T_cg) - (f - B.T.dot(lambda_cg))

In [ ]:
K.dot(T_feti) - (f - B.T@lambda_)

In [ ]:
B.dot(T_cg)

In [ ]:
T_feti

In [ ]:
eigval, eigvec = np.linalg.eig(F)

eigval

In [ ]:
F